In [ ]:
from graphein.protein.graphs import construct_graph
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.edges.distance import add_hydrogen_bond_interactions, add_ionic_interactions, add_peptide_bonds

# Create backbone graph
config = ProteinGraphConfig()
simple_graph = construct_graph(config, pdb_code="3eiy")

# Create backbone graph with additional interactions
edge_funcs = [add_hydrogen_bond_interactions, add_ionic_interactions, add_peptide_bonds]
config = ProteinGraphConfig(edge_construction_functions= edge_funcs)
complex_graph = construct_graph(config, pdb_code="3eiy")

## Plot

### Parameters
```
G: nx.Graph,
plot_title: Optional[str] = None,
figsize: Tuple[int, int] = (620, 650),
node_alpha: float = 0.7,
node_size_min: float = 20.0,
node_size_multiplier: float = 20.0,
label_node_ids: bool = True,
node_colour_map=plt.cm.plasma,
edge_color_map=plt.cm.plasma,
colour_nodes_by: str = "degree",
colour_edges_by: str = "type",
edge_alpha: float = 0.5,
```

In [ ]:
from graphein.protein.visualisation import plotly_protein_structure_graph

plotly_protein_structure_graph(
    G=simple_graph,
    plot_title="Simple Protein Structure (Backbone only)",
    colour_nodes_by="residue_number",
    colour_edges_by="kind",
    node_size_min=1
    )

In [ ]:
from graphein.protein.visualisation import plotly_protein_structure_graph

plotly_protein_structure_graph(
    G=complex_graph,
    plot_title="Residue level graph with Hydrogenbonds, Ionic interactions and Peptide bonds",
    colour_nodes_by="residue_number",
    colour_edges_by="kind",
    node_size_min=20,
    node_size_multiplier=1
    )

In [ ]:
from graphein.protein.edges.atomic import add_atomic_edges
config = ProteinGraphConfig(granularity='atom', edge_construction_functions=[add_atomic_edges])
g = construct_graph(config, pdb_code="3eiy")

# Create atomic graph
plotly_protein_structure_graph(
    g,
    plot_title="Atom level graph",
    colour_nodes_by="atom_type",
    colour_edges_by="kind",
    node_size_min=10,
    node_size_multiplier=1
    )